<style>
    div.container {
      max-width: 800px!important;
    }
</style>

# Spark SQL Basics

Spark SQL provides the means for working with structured data within Apache Spark.  Structured data is represented by the `DataFrame` abstraction (which is a type alias for `Dataset[Row]`), and we can act on them using familiar-looking SQL queries, or else the `DataFrame` API.  In this lesson, we cover `DataFrame` basics, including:

* creating `DataFrame`s in code
* creating `DataFrame`s from external sources (CSV, parquet, hive, PostgreSQL, etc.)
* manipulating and summarising `DataFrame`s using both SQL and the `DataFrame` API

## Preliminaries

This workbook makes use of the [Almond Scala kernel for Jupyter](https://almond.sh/).  To use Spark, we have to first add a few libraries to the classpath, which we can do as follows:

In [1]:
def init: Unit = {
  import ammonite.ops._
  val jars = ls! root/'opt/'spark/'jars |? (_.ext == "jar")
  jars.foreach(interp.load.cp(_))   
}

init

defined function init

Spark is also pretty verbose with respect to logging, so it can be useful to change the logging policy to de-clutter our outputs:

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.ERROR)

import org.apache.log4j.{Level, Logger}


And just to get it out of the way up front, we import a number of objects that we need throughout the rest of the document:

In [4]:
import org.apache.spark.sql._

import org.apache.spark.sql._

Finally, sometimes a code block will produce a large amount of output, some of which is unimportant, and so obfuscatory.  To hide this, we sometimes wrap things in an object like so:

```scala
object foo {
  val x = 1
  val y = 2
}

x + y
```

The object `foo` serves no functional purpose here other than to hide the interpreter output that results from the assignment of `x` and `y`.

## Creating a `SparkSession`

As of Spark 2.x, the usual method of interacting with Spark is by creating `SparkSession` to function as a single entrypoing.  In our case, we do this as follows:

In [5]:
val spark = SparkSession
  .builder
  .config("hive.metastore.uris","thrift://localhost:9083") 
  .config("spark.sql.warehouse.dir", "/data/hive/warehouse")
  .master("local[*]")
  .appName("Spark SQL Basics")
  .enableHiveSupport()
  .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


spark: SparkSession = org.apache.spark.sql.SparkSession@7a0ce2df

Here we explicitly configure our session to use Hive by setting values for `hive.metastore.uris` and `spark.sql.warehouse.dir`.  It is common for this to work automatically by configuration (via the file `hive-site.xml`), but this does not appear to the case in this context.  We also tell Spark to work in pseudo-distributed mode by setting `master` to `local[*]`.  There are various other configurations possible, but that's out of scope here.  See [Configuration - Spark 2.4.3 Documentation](https://spark.apache.org/docs/latest/configuration.html) for details.

When working with Spark SQL, it is very common to use the object `spark.sparkContext`.  So for convenience, we also assign this to a variable, commonly `sc`, as follows:

In [6]:
val sc = spark.sparkContext

sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@43bc9178

N.b. that we decreased the amount of debugging already in above.  We can also do this via the `SparkSession` object by running `sc.setLogLevel("ERROR")`, but then we'd still be subjected to the logging that occurs as a result of creating the `SparkSession` itself.

## Resilient Distributed Dataset (`RDD`)

As noted, `DataFrame` is the central data abstraction when working with structured data.  However, these build on an earlier abstraction called Resilient Distributed Datasets (`RDD`), and one will still have occasion to use these.  An `RDD` is essentially just a normal Scala collection that's been parallelised for use with Spark.  For example:

In [7]:
val beatles = Seq("John", "Paul", "Ringo", "George")
val distributedBeatles = spark.sparkContext.parallelize(beatles)

beatles: Seq[String] = List("John", "Paul", "Ringo", "George")
distributedBeatles: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[0] at parallelize at cmd6.sc:2

We can treat the resulting `RDD` in much the same way as the original collection, but the `RDD` will be worked on in parallel.  This means that when iterating over an `RDD` the order we will process entries will be unstable.  And because of this, certain operations that require a strictly ordered sequence, like `head` and `tail`, will not be available.  Regardless, the lack of a stable ordering is easily demonstrated:

In [8]:
// stable order for Scala Seq type
println(beatles.fold("")(_ + _))
println(beatles.fold("")(_ + _))

JohnPaulRingoGeorge
JohnPaulRingoGeorge


In [9]:
// but not for a parallelized collection
println(distributedBeatles.fold("")(_ + _))
println(distributedBeatles.fold("")(_ + _))

JohnRingoPaulGeorge
PaulJohnRingoGeorge


## Creating `DataFrame`s Programmatically

There are a number of ways we can create a `DataFrame`.  Since we just introduced `RDD`s, let us first demonstrate how we can create a `DataFrame` from an `RDD`: 

In [10]:
val data = Seq(
  ("George", "Harrison"),
  ("Ringo", "Starr"),
  ("John", "Lennon"),
  ("Paul", "McArtney")
)

data: Seq[(String, String)] = List(
  ("George", "Harrison"),
  ("Ringo", "Starr"),
  ("John", "Lennon"),
  ("Paul", "McArtney")
)

In [11]:
import spark.implicits._

sc
  .parallelize(data)
  .toDF("firstName", "lastName")
  .show

+---------+--------+
|firstName|lastName|
+---------+--------+
|   George|Harrison|
|    Ringo|   Starr|
|     John|  Lennon|
|     Paul|McArtney|
+---------+--------+



import spark.implicits._



## Creating `DataFrame`s from External Sources

`DataFrame` provides a single common interface for working with structured data.  Still, we can create a `DataFrame` from a number of different input types.  Here we cover several common scenarios.

## Hive

Apache Hive is a data warehouse that, among other things, provides SQL-like access to data stored on Hadoop. The `SparkContext` in this notebook has support for Hive enabled, and so we can query data in Hive tables using standard-looking SQL queries.  We first import `spark.sql` so we can write `sql(<query>)` instead of `spark.sql(<query>)`&ndash;not a huge convencience, but commonly done in other resources and code bases so we include it for consistency.  Then, we list the available databases:

In [12]:
import spark.sql
sql("show databases").show()

19/07/23 22:54:26 INFO metastore: Trying to connect to metastore with URI thrift://localhost:9083
19/07/23 22:54:26 INFO metastore: Connected to metastore.


+------------+
|databaseName|
+------------+
|     default|
|  nycflights|
+------------+



import spark.sql


The `nycflights` database is pre-populated from [Bureau of Transportation Statistics](https://www.transtats.bts.gov/) data, as provided by the [nycflights13](https://github.com/hadley/nycflights13) R package.  We can list the available tables as follows:

In [13]:
sql("use nycflights")
sql("show tables").show

+----------+---------+-----------+
|  database|tableName|isTemporary|
+----------+---------+-----------+
|nycflights| airlines|      false|
|nycflights| airports|      false|
|nycflights|  flights|      false|
|nycflights|   planes|      false|
|nycflights|  weather|      false|
+----------+---------+-----------+



res12_0: DataFrame = []

So, to pull the whole `airlines` table, for example, and save the results in a `DataFrame` called `airlines`, we just run the following:

In [14]:
val airlines: DataFrame = sql("SELECT * FROM nycflights.airlines")
airlines.show
airlines.schema.foreach(println)

+-------+--------------------+
|carrier|                name|
+-------+--------------------+
|     9E|   Endeavor Air Inc.|
|     AA|American Airlines...|
|     AS|Alaska Airlines Inc.|
|     B6|     JetBlue Airways|
|     DL|Delta Air Lines Inc.|
|     EV|ExpressJet Airlin...|
|     F9|Frontier Airlines...|
|     FL|AirTran Airways C...|
|     HA|Hawaiian Airlines...|
|     MQ|           Envoy Air|
|     OO|SkyWest Airlines ...|
|     UA|United Air Lines ...|
|     US|     US Airways Inc.|
|     VX|      Virgin America|
|     WN|Southwest Airline...|
|     YV|  Mesa Airlines Inc.|
+-------+--------------------+

StructField(carrier,StringType,true)
StructField(name,StringType,true)


airlines: DataFrame = [carrier: string, name: string]

And we can also run familiar-looking queries such as joins and grouped summaries.  For example

In [15]:
val flightsByCarrier = sql("""
select 
  name, sum(1) as num_flights 
from 
  (
    select 
      airlines.name, flights.* 
    from 
      nycflights.airlines 
    inner join 
      nycflights.flights 
    on 
      airlines.carrier = flights.carrier
  ) a
group by 
  name 
order by 
  name
""")

flightsByCarrier.show()

+--------------------+-----------+
|                name|num_flights|
+--------------------+-----------+
|AirTran Airways C...|       3260|
|Alaska Airlines Inc.|        714|
|American Airlines...|      32729|
|Delta Air Lines Inc.|      48110|
|   Endeavor Air Inc.|      18460|
|           Envoy Air|      26397|
|ExpressJet Airlin...|      54173|
|Frontier Airlines...|        685|
|Hawaiian Airlines...|        342|
|     JetBlue Airways|      54635|
|  Mesa Airlines Inc.|        601|
|SkyWest Airlines ...|         32|
|Southwest Airline...|      12275|
|     US Airways Inc.|      20536|
|United Air Lines ...|      58665|
|      Virgin America|       5162|
+--------------------+-----------+



flightsByCarrier: DataFrame = [name: string, num_flights: bigint]

## CSV

We can read a variety of external formats via the `SparkContext`, and the pattern is largely the same from format to format.  In this case, we read the famous iris dataset which has been saved locally as a csv file as `/data/csv/iris.csv`.  To import it:

In [16]:
val iris = spark
  .read
  .format("csv")
  .option("header", "true")
  .option("delimiter", ",")
  .option("inferSchema", "true")
  .load("/data/csv/iris.csv")

iris: DataFrame = [Sepal.Length: double, Sepal.Width: double ... 3 more fields]

In [17]:
iris.limit(5).show

+------------+-----------+------------+-----------+-------+
|Sepal.Length|Sepal.Width|Petal.Length|Petal.Width|Species|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| setosa|
|         4.9|        3.0|         1.4|        0.2| setosa|
|         4.7|        3.2|         1.3|        0.2| setosa|
|         4.6|        3.1|         1.5|        0.2| setosa|
|         5.0|        3.6|         1.4|        0.2| setosa|
+------------+-----------+------------+-----------+-------+



Of course, if we are working in a 'Big Data' environment, we might expect that our file is saved in Hadoop, rather than locally on disk.  For example, we could copy our file to the Hadoop filesystem by running the following:

```bash
$ hadoop hdfs -mkdir /data
$ hadoop hdfs -mkdir /data/csv
$ hadoop hdfs -put /data/csv/iris.csv /data/csv/
```

The Hadoop filesystem in our case is `hdfs://localhost:9000` (configured in `/opt/hadoop/conf/core-site.xml` via the `fs.defaultFS` property), and in this case we would just change the import as follows:

```scala
val iris = spark
  .read
  .format("csv")
  .option("header", "true")
  .option("delimiter", ",")
  .option("inferSchema", "true")
  .load("hdfs://localhost:9000/user/root/iris.csv")
```

## Parquet and Optimized Row Columnar (ORC)

Parquet and ORC are popular columnar formats&ndash;parquet more so for Spark, and ORC more so for Hive.  Because data is stored in columns, compression algorithms appropriate for specific columns can be applied, and so the formats generally have good to excellent compression performance.  As an example, the airlines database used above is 53.8MB on disk when stored as CSV, but 7.5MB and 6.0MB when stored as ORC and parquet, respectively.  They also tend to perform very well in read applications like grouped aggregated, though don't fare as well in write applications.  Either way, we could repreduce the example above where we calculated the number of flights by airline as follows:

In [31]:
val airlines = spark
  .read
  .format("parquet")
  .load("/data/parquet/nycflights/airlines/")

val flights = spark
  .read
  .format("orc")
  .load("/data/orc/nycflights/flights/")

airlines: DataFrame = [carrier: string, name: string]
flights: DataFrame = [year: int, month: int ... 17 more fields]

In [33]:
import org.apache.spark.sql.functions._

airlines.as("airlines")
  .join(flights.as("flights"), col("airlines.carrier") === col("flights.carrier"), "inner")
  .groupBy("name")
  .count
  .withColumnRenamed("count", "num_flights")
  .orderBy("name")
  .show

+--------------------+-----------+
|                name|num_flights|
+--------------------+-----------+
|AirTran Airways C...|       3260|
|Alaska Airlines Inc.|        714|
|American Airlines...|      32729|
|Delta Air Lines Inc.|      48110|
|   Endeavor Air Inc.|      18460|
|           Envoy Air|      26397|
|ExpressJet Airlin...|      54173|
|Frontier Airlines...|        685|
|Hawaiian Airlines...|        342|
|     JetBlue Airways|      54635|
|  Mesa Airlines Inc.|        601|
|SkyWest Airlines ...|         32|
|Southwest Airline...|      12275|
|     US Airways Inc.|      20536|
|United Air Lines ...|      58665|
|      Virgin America|       5162|
+--------------------+-----------+



import org.apache.spark.sql.functions._



In [25]:
def timeit[T](block: =>T): (T, Double) = {
  val startTime = System.currentTimeMillis()
  val res: T = block
  (res, System.currentTimeMillis() - startTime)
}

defined function timeit